# Import Libraries

In [1]:
from utils_metrics import get_entities_bio, f1_score, precision_score, recall_score, classification_report
from transformers import BartForConditionalGeneration, BartTokenizer, BartConfig
import torch
import time
import math
import random

ModuleNotFoundError: ignored

In [ ]:
# Uncomment if trained on GCP
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Uncomment if trained locally
# device = torch.device("mps")
# print(device)

In [ ]:
# CUDA_LAUNCH_BLOCKING=1
device

# Class and Functions

In [ ]:
class InputExample():
    def __init__(self, words, labels):
        self.words = words
        self.labels = labels

In [ ]:
def template_entity(words, input_TXT, start):
    # input text -> template
    words_length = len(words)
    words_length_list = [len(i) for i in words]
    input_TXT = [input_TXT]*(5*words_length)

    input_ids = tokenizer(input_TXT, return_tensors='pt')['input_ids']
    model.to(device)
    template_list = [" is a location entity .", " is a person entity .", " is an organization entity .",
                     " is an other entity .", " is not a named entity ."]
    entity_dict = {0: 'LOC', 1: 'PER', 2: 'ORG', 3: 'MISC', 4: 'O'}
    temp_list = []
    for i in range(words_length):
        for j in range(len(template_list)):
            temp_list.append(words[i]+template_list[j])

    output_ids = tokenizer(temp_list, return_tensors='pt', padding=True, truncation=True)['input_ids']
    output_ids[:, 0] = 2
    output_length_list = [0]*5*words_length


    for i in range(len(temp_list)//5):
        base_length = ((tokenizer(temp_list[i * 5], return_tensors='pt', padding=True, truncation=True)['input_ids']).shape)[1] - 4
        output_length_list[i*5:i*5+ 5] = [base_length]*5
        output_length_list[i*5+4] += 1

    score = [1]*5*words_length
    with torch.no_grad():
        output = model(input_ids=input_ids.to(device), decoder_input_ids=output_ids[:, :output_ids.shape[1] - 2].to(device))[0]
        for i in range(output_ids.shape[1] - 3):
            # print(input_ids.shape)
            logits = output[:, i, :]
            logits = logits.softmax(dim=1)
            # values, predictions = logits.topk(1,dim = 1)
            logits = logits.to('cpu').numpy()
            # print(output_ids[:, i+1].item())
            for j in range(0, 5*words_length):
                if i < output_length_list[j]:
                    score[j] = score[j] * logits[j][int(output_ids[j][i + 1])]

    end = start+(score.index(max(score))//5)
        # score_list.append(score)
    return [start, end, entity_dict[(score.index(max(score))%5)], max(score)] #[start_index,end_index,label,score]

In [ ]:
def prediction(input_TXT):
    input_TXT_list = input_TXT.split(' ')

    entity_list = []
    for i in range(len(input_TXT_list)):
        words = []
        for j in range(1, min(9, len(input_TXT_list) - i + 1)):
            word = (' ').join(input_TXT_list[i:i+j])
            words.append(word)

        entity = template_entity(words, input_TXT, i) #[start_index,end_index,label,score]
        if entity[1] >= len(input_TXT_list):
            entity[1] = len(input_TXT_list)-1
        if entity[2] != 'O':
            entity_list.append(entity)
    i = 0
    if len(entity_list) > 1:
        while i < len(entity_list):
            j = i+1
            while j < len(entity_list):
                if (entity_list[i][1] < entity_list[j][0]) or (entity_list[i][0] > entity_list[j][1]):
                    j += 1
                else:
                    if entity_list[i][3] < entity_list[j][3]:
                        entity_list[i], entity_list[j] = entity_list[j], entity_list[i]
                        entity_list.pop(j)
                    else:
                        entity_list.pop(j)
            i += 1
    label_list = ['O'] * len(input_TXT_list)

    for entity in entity_list:
        label_list[entity[0]:entity[1]+1] = ["I-"+entity[2]]*(entity[1]-entity[0]+1)
        label_list[entity[0]] = "B-"+entity[2]
    return label_list

In [ ]:
def cal_time(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

# Load Model and Tokenizer

In [ ]:
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')
# input_TXT = "Japan began the defence of their Asian Cup title with a lucky 2-1 win against Syria in a Group C championship match on Friday ."

# Pretrained model
# model = BartForConditionalGeneration.from_pretrained('facebook/bart-large')

# Self-trained model
model = BartForConditionalGeneration.from_pretrained('./outputs/best_model')

model.eval()

model.config.use_cache = False
# input_ids = tokenizer(input_TXT, return_tensors='pt')['input_ids']
# print(input_ids)


In [ ]:
# Source
# device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu") # Source

# Modified by Fina
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
score_list = []
# file_path = './data/conll2003/test.txt'
# file_path = './data/MIT_movie/test.txt'
file_path = './data/MIT_restaurant/test.txt'

guid_index = 1
examples = []

In [ ]:
def open_conLL2003(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        words = []
        labels = []
        for line in f:
            if line.startswith("-DOCSTART-") or line == "" or line == "\n":
                if words:
                    examples.append(InputExample(words=words, labels=labels))
                    words = []
                    labels = []
            else:
                # splits = line.split(" ")
                splits = line.split("\t")       # Modified by Alfina
                words.append(splits[0])
                if len(splits) > 1:
                    labels.append(splits[-1].replace("\n", ""))
                else:
                    # Examples could have no label for mode = "test"
                    labels.append("O")
        if words:
            examples.append(InputExample(words=words, labels=labels))

In [ ]:
def open_MIT(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        words = []
        labels = []
        for line in f:
            if line.startswith("-DOCSTART-") or line == "" or line == "\n":
              if words:
                examples.append(InputExample(words=labels, labels=words))  # Swap words and labels
                words = []
                labels = []
            else:
              splits = line.split("\t")    # Modified by Alfina
              words.append(splits[0])
              if len(splits) > 1:
                labels.append(splits[-1].replace("\n", ""))
              else:
                labels.append("O")
            if words:
                examples.append(InputExample(words=labels, labels=words))  # Swap words and labels

# Precondition Test Dataset

In [ ]:
if file_path[7] == 'c':
    open_conLL2003(file_path)
elif file_path[7] == 'M':
    open_MIT(file_path)

In [ ]:
# For training purpose; reducing the number of dataset

# Get random
examples_random_sample = random.sample(examples, 50)
num_01 = len(examples_random_sample)

In [ ]:
# # Debug
# for example in examples_random_sample:
#     print(example.words)
#     print(example.labels)
#     print("---------------")

# Inference

In [ ]:
trues_list = []
preds_list = []
str = ' '
num_point = 0
start = time.time()

for example in examples_random_sample:
    sources = str.join(example.words)
    preds_list.append(prediction(sources))
    trues_list.append(example.labels)
    print('%d/%d (%s)'%(num_point+1, num_01, cal_time(start)))
    print('Pred:', preds_list[num_point])
    print('Gold:', trues_list[num_point])
    num_point += 1


1/50 (0m 0s)
Pred: ['O', 'O', 'O', 'O', 'O', 'B-LOC', 'O', 'O', 'O']
Gold: ['O', 'O', 'O', 'O', 'O', 'B-Cuisine', 'O', 'B-Location', 'I-Location']
2/50 (0m 1s)
Pred: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Gold: ['O', 'O', 'O', 'B-Rating', 'O', 'O', 'O', 'B-Location', 'I-Location', 'O', 'B-Price', 'I-Price', 'I-Price', 'I-Price']
3/50 (0m 2s)
Pred: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Gold: ['O', 'O', 'O', 'O', 'O', 'B-Hours', 'O', 'O', 'B-Amenity']
4/50 (0m 4s)
Pred: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Gold: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Amenity', 'I-Amenity', 'O', 'O', 'B-Dish', 'O', 'O', 'B-Location', 'I-Location', 'I-Location', 'O', 'O', 'O', 'O']
5/50 (0m 5s)
Pred: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOC']
Gold: ['O', 'O', 'O', 'O', 'O', 'B-Location', 'I-Location', 'I-Location', 'I-Location', 'I-Location']
6/50 (0m 5s)
Pred: ['O', 'O', 'O', 'O', '

In [ ]:
# F1-score only
true_entities = get_entities_bio(trues_list)
pred_entities = get_entities_bio(preds_list)
results = {
    "precision": precision_score(true_entities, pred_entities),
    "recall": recall_score(true_entities, pred_entities),
    "f1": f1_score(true_entities, pred_entities)

}
print("--------------------------")
print("Precision:  ",results["precision"])
print("Recall:  ",results["recall"])
print("F1-score:  ",results["f1"])

nb_correct:  124
nb_pred:  27
nb_true:  97
--------------------------
Precision:   4.592592592592593
Recall:   1.2783505154639174
F1-score:   2.0


# Write Output Files

In [ ]:
for num_point in range(len(preds_list)):
    preds_list[num_point] = ' '.join(preds_list[num_point]) + '\n'
    trues_list[num_point] = ' '.join(trues_list[num_point]) + '\n'
with open('./pred.txt', 'w') as f0:
    f0.writelines(preds_list)
with open('./gold.txt', 'w') as f0:
    f0.writelines(trues_list)